# Section6：物体検知・セグメンテーション

## 物体認識のタスク

入力データ：画像


| 予測方法 | 出力結果 |
| :---- | :--- |
| 分類 （Classification） | （画像に対し単一または複数の）クラスラベル |
| 物体検知 （Object Detection） | Bounding Box [bbox / BB] |
| 意味領域分割 （Semantic Segmentation） | （各ピクセルに対し単一の）クラスラベル |
| 個体領域分割 （Instance Segmentation） | （各ピクセルに対し単一の）クラスラベル |

#### 分類 
- 物体を検知するのみである。
- 位置に意味はない

#### 物体検知
- どこにあるか
- 四角の箱で位置を示す

#### 意味領域分割
- どこにあるか
- ピクセル毎に同じ色をつける。関係のない部分は白黒にする。

#### 個体領域分割
- どこにあるか
- ピクセル毎に同じ色をつける。関係のない部分は白黒にする。
- 同じ物体でも、事なる個体には異なる色をつける

## 代表的なデータセット

物体検出に用いられたデータセット
テストをする際に、比較可能なデータとするため、データセットの特徴を把握しておく必要がある。

| 名称 | クラス | Train+Val | BOX/画像 | 
| :---- | :---: | :---: | :---: |
| VOC12 | 20 | 11,540 | 2.4 |
| ILSVRC17 | 200 | 476,668 | 1.1 |
| MS COCO18 | 80 | 123,287 | 7.3 |
| OICOD18 | 500 | 1,743,042 | 7.0 |

「BOX/画像」は、1枚の画像あたりにいくつ物体があるかの平均
１枚の画像あたりに物体数が少ないと日常生活の画像ではない

## 評価指標

### Confusion Matrix

|  | Positive | Nagative |
| :---: | :---: | :---: | 
| Positive | True<br>Positive | False<br>Nagative |
| Nagative | False<br>Positive | True<br>Nagative | 

<br>

\begin{aligned}
    Precision &= \frac{TP}{TP+FP} \\[8px]
    Recall &= \frac{TP}{TP+FN}
\end{aligned}

### IoU （Intersection over Union）

物体検出においては、クラスラベルだけでなく、物体位置の予測精度も評価したい


\begin{aligned}
    IoU = \frac{TP}{TP+FP+FN}
\end{aligned}

### AP （Average Precision）


\begin{aligned}
    AP = \int_0^1 P(R)dR
\end{aligned}

### mAP （mean Average Precision）


\begin{aligned}
    mAP = \frac{1}{C} \sum_{i=1}^C AP_i
\end{aligned}

### MS COCO


\begin{aligned}
    mAP_{COCO} = \frac{mAP_{0.5} + mAP_{0.55} +  \ldots  + mAP_{0.95} }{10}
\end{aligned}

### FPS （Flames per Second）

検出精度に加え、検出速度も問題となる



### SSD （Single Shot Detector）

#### 概要
1. Default BOXを用意する
1. Default BOXを変形して、Predicted BB とする。

#### 特徴マップからの出力
`出力サイズ`：#Class + 4 \
※後ろの４は、オフセット項

マップ中の各特徴量に$k$個のDefault Boxを用意するとき,  \
`出力サイズ`：$k$ (#Class + 4) 

更に, 特徴マップのサイズが$m \times n$であるとすれば,  \
`出力サイズ`：$k$ (#Class + 4) $mn$

### 多数のDefault Boxを用意したことで生ずる問題への対処
- Non-Maximum Suppression \
ひとつの物体しか映っていないにも関わらず、複数のBBが発生する \
　⇒IoUを計算し、被っている場合は削除する。

- Hard Negative Mining \
背景と判断されるBBが発生する可能性がある。

※RCNNでも既に用いられており初出の処理ではない
※他にも, 原著論文では学習上の工夫としてDefault Box のアスペクト比の選び方やData Augmentationについても触れられている

# Semantic Segmentationの概略

## 問題
Convolutionをかけると、解像度が下がる。

### Up-samplingなんて面倒なことが必要になるなら、そもそもPoolingしなければ良いのでは？

⇒正しく認識するためには受容野にある程度の大きさが必要であった。
　受容野を広げる典型：①深いConv.層 ②プーリング（ストライド）

## FCN（Fully Convolutional Network）の基本アイディア

全結合層の部分をConvolution層に変更すると、ヒートマップになる。


### Deconvolution/Transposed convolution
アップサンプリングの手法

※逆畳み込みと呼ばれることも多いが畳み込みの逆演算ではないことに注意 \
　→当然, poolingで失われた情報が復元されるわけではない